In [ ]:
import trackpy as tp
import os
import pandas as pd
import numpy as np
import numba
import matplotlib.pyplot as plt
import cv2
import math
import warnings
warnings.filterwarnings(action='once')

from rotational_drift_subtraction import drift_subtract

In [ ]:
folderlist = [r'G:\Shared drives\Horowitz Lab Notes\Rebecca Dalphin and Alex Axton\Analysis\2023-05-27',
             r'G:\Shared drives\Horowitz Lab Notes\Rebecca Dalphin and Alex Axton\Analysis\2023-05-28']

# scaling, measured in microns per pixel
#scaling = 330 / 1247.96 # 20x1.0, measured 2021-06-17
#scaling = 220 / 1250.04 # 20x1.5, measured 2021-06-17
scaling = 150 / 1127.54 # 40x1.0, measured 2021-06-16
#scaling = 100 / 1130.61 # 40x1.5, measured 2021-06-16
#scaling = 80 / 914.92 # 60x1.0, measured 2021-05-28
#scaling = 60 / 1031.07 # 60x1.5, measured 2021-05-28

In [ ]:
df_list = []
filename_list = []
fps_list = []

"""
Note about file names
unfiltered.pkl = every particle trajectory, immediately after it was linked.
.pkl = filtered to remove stubs
nodrift.pkl = drift has been subtracted
"""



for folder in folderlist:
    os.chdir(folder)
    # Iterate over files in the folder
    for filename in os.listdir(folder):
        if filename.endswith('control_tracer.pkl'):
            print(filename)
            df_list.append(pd.read_pickle(filename))
            filename_list.append(os.path.join(folder,filename))
            
#filename_list

In [ ]:
import av


def get_fps_from_avi(avi_file): # avi_file is a file path
    try:
        video = cv2.VideoCapture(avi_file)
        fps = video.get(cv2.CAP_PROP_FPS)
        video.release()
    except:
        import av
        container = av.open(avi_file)
        fps = container.streams.video[0].average_rate
        container.close()
    return float(fps)

os.chdir(r'G:\Shared drives\Horowitz Lab Notes\Rebecca Dalphin and Alex Axton')
for filename in os.listdir(r'G:\Shared drives\Horowitz Lab Notes\Rebecca Dalphin and Alex Axton'):
    #print(filename)
    if filename.endswith('.avi'):
        fps = get_fps_from_avi(filename)
        print(fps)


In [ ]:
df_list_nodrift = []
for df in df_list:
    _, df_nodrift = drift_subtract(tracer = df )
    df_list_nodrift.append(df_nodrift)

In [ ]:
debug = False

if debug and len(filename_list) == len(df_list_nodrift):
    df_list_nodrift_orig = df_list_nodrift.copy()
    df_list_nodrift = df_list_nodrift[:4]

In [ ]:
## Calculate probability distribution function

maxlagtime = 30 # in number of frames
vanhove_max_x = 5
binwidth = 0.05
figsize = [8,8]


warnings.filterwarnings("ignore", category=RuntimeWarning)
binsequence=np.arange(-vanhove_max_x,vanhove_max_x,binwidth)
trackshistlistlist = []
kurtosisdflist = []
lagtimelist =[]
numplots = maxlagtime
i = 0
fig, axs = plt.subplots(int(math.ceil(numplots/5)), 5, figsize = figsize, dpi=300)
#plt.sca(axs.flat[0])

fig2, ax2 = plt.subplots(1,1, figsize = figsize, dpi=300)


"""
Normalizes histogram data so that the sum of probabilities is one.

@param histdata - the starting histogram

@return the normalized histogram of probabilites
"""
def manualnorm(histdata):
     return (1/(histdata.sum()*binwidth))*histdata
    
for vanhove_lagtime in range(maxlagtime):
    lagtimelist.append(vanhove_lagtime/fps)
    
for (file,df) in zip(filename_list,df_list_nodrift):
    print(file)
    tracksbyframe = df.set_index(['frame','particle'])['x'].unstack()
    trackshistlist = []
    klist = []
    tlist = []
    for vanhove_lagtime in range(maxlagtime):
        #print('vanhove_lagtime:', vanhove_lagtime, 'frames, or', vanhove_lagtime/fps, 'sec' )
        plt.sca(axs.flat[vanhove_lagtime])
        plt.title("{:.2f} ms".format(1000 * vanhove_lagtime / fps))
        # calculate vanhove / probability distribution function
        trackshist = manualnorm(tp.motion.vanhove(tracksbyframe, lagtime = vanhove_lagtime, mpp =
                scaling,bins = binsequence, ensemble = True))
        trackshist.plot(ax=axs.flat[vanhove_lagtime])
        trackshistlist.append(trackshist)
        
        #plt.sca(axs2.flat[i])
        k = trackshist.kurtosis()
        
        klist.append(k)
        tlist.append(vanhove_lagtime/fps)
        #plt.scatter(x=vanhove_lagtime/fps, y = k)
        
    plt.sca(ax2)
    kurtosisdf = pd.DataFrame({'time (s)': tlist, 'kurtosis': klist, 'filename': file})
    plt.xlabel('lag time (s)')
    plt.ylabel('excess kurtosis')
    kurtosisdf[['time (s)', 'kurtosis']].plot(label = file)

    trackshistlistlist.append(trackshistlist)
    kurtosisdflist.append(kurtosisdf)
    i = i+1
    
plt.show()

# Re-enable the warning
warnings.filterwarnings("default", category=RuntimeWarning)

In [ ]:
trackshist.plot()